### API 키 불러오기

In [1]:
import os
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
kakao_api_key = config['KAKAO_MAP']['KAKAO_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

In [4]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate, load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

### Get Stage Analyzer Prompt

In [5]:
stage_analyzer_inception_prompt = load_prompt("./templates/stage_analyzer_inception_prompt_template.json")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)
stage_analyzer_chain = LLMChain(
    llm=llm,
    prompt=stage_analyzer_inception_prompt, 
    verbose=True, 
    output_key="stage_number")

### Get User Response candidates Generation Prompt

In [6]:
user_response_prompt = load_prompt("./templates/user_response_prompt.json")
# 랭체인 모델 선언, 랭체인은 언어모델과 프롬프트로 구성됩니다.
llm = ChatOpenAI(model='gpt-4', temperature=0.5)
user_response_chain = LLMChain(
    llm=llm,
    prompt=user_response_prompt, 
    verbose=True, # 과정을 출력할지
    output_key="user_responses"
)

### Load wine database json

In [121]:
df = pd.read_json('./data/unified_wine_data.json', encoding='utf-8', lines=True)
# df = pd.read_json('./data/wine_nara.json', encoding='utf-8', lines=True)

In [104]:
df.tail()

,url,site_name,price,name,en_name,img_url,body,acidity,tannin,sweetness,alcohol,wine_type,country,grape,rating,pickup_location,vivino_link,flavor_description,pairing
1565,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,"55,000",나파 컷 까베르네 소비뇽,Napa Cut Cabernet Sauvignon NAPA CUT,./data/wine_and_more/img/나파_컷_까베르네_소비뇽.png,,,,,,,,,,"와인앤모어 논현점,와인앤모어 마포공덕점,와인앤모어 서울숲점,와인앤모어 성수역점,와인...",,"나파밸리 가성비 스테이크 와인 아름다운 짙은 루비색을 띄고 있으며, 잔에 와인을 따...","소고기 스테이크, 바비큐"
1566,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,"219,900",실버 오크 알렉산더 밸리,Silver Oak Alexander Valley SILVEROAK CABERNET...,./data/wine_and_more/img/실버_오크_알렉산더_밸리.png,,,,,,,,,,"와인앤모어 동판교점,와인앤모어 마포공덕점,와인앤모어 서울숲점,와인앤모어 센텀점,와인...",,와인스피릿이 선정한 NO.1 까베르네 쇼비뇽! 24월간 American Oak 에서...,"포크찹, 비프 스테이크, 프라임 립"
1567,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,"36,000",[6월행사] [온라인단독] 라피스 루나 진판델,Lapis Luna Zinfandel,./data/wine_and_more/img/[6월행사]_[온라인단독]_라피스_루나...,,,,,,,,,,"와인앤모어 AK광명점,와인앤모어 광교점,와인앤모어 광화문점,와인앤모어 군자역점,와인...",,"진판델의 당도, 산지오베제의 조화를 통한 부드러운 레드와인 블루베리, 블랙베리, 후...",타코 파니니
1568,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,"150,000","릿지, 릿톤 진판델",Ridge,"./data/wine_and_more/img/릿지,_릿톤_진판델.png",,,,,,,,,,와인앤모어 논현점,,", Lytton Springs Zinfandel BOOKS 파리의 심판 30주년 시...",소고기 양고기 치즈
1569,https://www.wineandmore.co.kr/goods/goods_view...,wine_and_more,"22,000",브라운 브라더스 모스카토 로사,R,./data/wine_and_more/img/브라운_브라더스_모스카토_로사.png,,,,,,,,,,"와인앤모어 AK광명점,와인앤모어 광교점,와인앤모어 동판교점,와인앤모어 삼성1호점,와...",,모스카토 로사 Brown Brothers Moscato Rosa MOSCATO 사랑...,부드러운 치즈 과일 디저트


### Prepare Langchain Tool

#### Tool1: Wine database 1

In [122]:
df['page_content'] = ''
columns = ['name', 'pairing']
for column in columns:
    if column != 'page_content':
        df['page_content'] += column + ':' + df[column].astype(str) + ','

In [123]:
columns = ['rating', 'price', 'body', 'sweetness', 'alcohol', 'acidity', 'tannin']
for idx in df.index:
    for column in columns:
        if type(df[column][idx]) == str:
            df[column][idx] = df[column][idx].replace(',', '')
        df[column][idx] = float(df[column][idx]) if df[column][idx] != '' else -1

In [124]:
loader =DataFrameLoader(data_frame=df, page_content_column='page_content')
docs = loader.load()
embeddings = OpenAIEmbeddings()

아래는 wine database1에 metadata_field Attribute이다. 아래를 기준으로 서치를 진행하게 된다.

In [125]:
metadata_field_info = [
       AttributeInfo(
        name="price",
        description="The price of the wine",
        type="int",
    ),
      AttributeInfo(
        name="rating", 
        description="1-5 rating for the wine", 
        type="float"
    ),
    AttributeInfo(
        name="wine_type", 
        description="The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'", 
        type="string"
    ),
    AttributeInfo(
        name="country", 
        description="The country of wine. It can be '기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주'", 
        type="string"
    ),
    AttributeInfo(
        name="body",
        description="1-5 rating for the body of wine",
        type="int",
    ),
    AttributeInfo(
        name="sweetness",
        description="1-5 rating for the sweetness of wine",
        type="int",
    ),
    AttributeInfo(
        name="alcohol",
        description="1-5 rating for the alcohol of wine",
        type="int",
    ),
]

In [126]:
wine_vectorstore = Chroma.from_documents(docs, embeddings)
document_content_description = "Database of a wine, wine name and food pairing in query, and wine type, country, price, rating, body, sweetness, alcohol in filter."
llm = OpenAI(temperature=0)
wine_retriever = SelfQueryRetriever.from_llm(
    llm, wine_vectorstore, document_content_description, metadata_field_info, verbose=True
)  # Added missing closing parenthesis

In [85]:
wine_vectorstore.similarity_search('육류', k=5, filter={"rating": {"$gt": 2.5}})

[Document(page_content='name:락베어 샤르도네,pairing:구운 연어 요리, 돼지 고기 요리,', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=04D451', 'site_name': 'winenara', 'price': 39000.0, 'name': '락베어 샤르도네', 'en_name': 'ROCKBARE CHARDONNAY', 'img_url': 'https://www.winenara.com/uploads/product/550/f3117d5ce81bfd1d9adf3b43a656513c.png', 'body': 3.0, 'acidity': -1, 'tannin': -1, 'sweetness': -1.0, 'alcohol': -1.0, 'wine_type': '화이트', 'country': '호주', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/US-CA/en/rock-bare-adelaide-hills-chardonnay/w/7568271', 'flavor_description': '시트러스 꽃잎, 천도 복숭아와 그린 망고향이 크리미한 캐슈넛의 향미와 아름답게 조화를 이룹니다. 입 안에선 시트러스 계열 과실과 허니듀 멜론의 신선한 맛이 매력적 입니다. 수개월간 바또나즈 작업으로 가미된 밸런스가 매끈한 감촉을 느낄 수 있게 해줍니다. 생강 스파이시함, 크리미한 피니시가 프렌치 오크 숙성을 만나 고조됩니다.', 'pairing': '구운 연어 요리, 돼지 고기 요리'}),
 Document(page_content='name:끌로 모가도르,pairing:구운 채소, 로스트 치킨,', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03R

In [86]:
wine_retriever.get_relevant_documents('price gt 0 lt 200000, rating gt 3.0')

c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='price', value=0), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='price', value=200000), Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=3.0)]) limit=None


[Document(page_content='name:비냐 조잘 말라예토,pairing:,', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03Q792', 'site_name': 'winenara', 'price': 55000.0, 'name': '비냐 조잘 말라예토', 'en_name': 'VINA ZORZAL MALAYETO', 'img_url': 'https://www.winenara.com/uploads/product/550/81b4138d23cd8bc759ba7a69cebc13a6.png', 'body': 3.0, 'acidity': -1, 'tannin': -1, 'sweetness': -1.0, 'alcohol': -1.0, 'wine_type': '레드', 'country': '스페인', 'grape': '', 'rating': 3.7, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/US-CA/en/vina-zorzal-malayeto/w/3384125', 'flavor_description': '블랙베리, 블랙 커런트 등 검은 과일의 신선하고 짙은 아로마가 코를 간지럽히며 옅은 블랙페퍼와 흙 뉘앙스를 함께 보여줍니다. 이른 수확은 신선한 느낌을 더합니다. 중간 정도의 바디감과 함께 검은 과일 뉘앙스를느낄 수 있습니다. 부드러운 탄닌, 산도의 적절한 조화로 긴 여운이 느껴집니다.', 'pairing': ''}),
 Document(page_content='name:더 페데럴리스트 샤르도네,pairing:,', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=04D405', 'site_name': 'winenara', 'price': 50000.0, 'name': '더 페데럴리스트 샤르도네', '

#### Tool2: Wine bar database

In [37]:
df = pd.read_json('./data/wine_bar.json', encoding='utf-8', lines=True)

In [38]:
df['page_content'] = ''
columns = ['review']
for column in columns:
    if column != 'page_content':
        df['page_content'] += df[column].astype(str) + ','

In [39]:
df = df.drop(columns=['review'])

In [40]:
df.head()

,url,name,rating,address,phone,price_range,parking,opening_hours,holidays,menu,img_urls,district,page_content
0,https://www.mangoplate.com/restaurants/Ha-RjMR...,미도림,3.9,서울특별시 성동구 왕십리로 108 3F\n지번 서울시 성동구 성수동1가 656-10...,02-469-8486,만원-2만원,주차공간없음,월-금: 16:00 - 24:00\n토-일: 13:00 - 01:00,,"취나물파스타:14,000원,부추교자:12,000원,더덕약고추장 곁들인 새우전변:12...",https://mp-seoul-image-production-s3.mangoplat...,성동구,['오랫동안 가고 싶었던 와인바 미도림. 성수동 와인바하면 대표적인 곳이 되었네요....
1,https://www.mangoplate.com/restaurants/dU0u62b...,사브서울,4.6,서울특별시 강남구 논현로175길 6 B1\n지번 서울시 강남구 신사동 580-5 B1,02-512-4939,3만원-4만원,주차공간없음,18:00 - 24:00,일,"대왕문어 카르파쵸:15,000원,비트앤엔다이브:17,000원,라비올로디우보:28,0...",https://mp-seoul-image-production-s3.mangoplat...,강남구,['맛집을 좋아하긴 하지만 그렇다고 예약이 치열하거나 웨이팅이 심한 곳은 별로 탐하...
2,https://www.mangoplate.com/restaurants/9Ldy8KH...,바코드,4.3,서울특별시 서대문구 연세로9길 26\n지번 서울시 서대문구 창천동 52-38,010-9029-6369,만원-2만원,주차공간없음,19:00 - 03:00,,,https://mp-seoul-image-production-s3.mangoplat...,서대문구,['간단평: 옛날에 가서 기억이 잘안나긴 하는데 칵테일바인데 음식이 꽤 괜찮아서 놀...
3,https://www.mangoplate.com/restaurants/1N5tENR...,와인소셜,4.1,서울특별시 강남구 압구정로46길 71-1 2F\n지번 서울시 강남구 신사동 648-...,0507-1445-0604,3만원-4만원,,,,"와인 5잔+플레이트:45,000원,",https://mp-seoul-image-production-s3.mangoplat...,강남구,"['Course, B / Olive / Tapemade with Baguette',..."
4,https://www.mangoplate.com/restaurants/l4SRQIF...,수도원,4.6,서울특별시 종로구 동숭3길 16 B1\n지번 서울시 종로구 동숭동 50-11 B1,02-747-1933,만원-2만원,유료주차 가능,월-금: 17:00 - 01:00\n토-일: 16:00 - 01:00,,"올드 라스푸틴 (330ml):13,000원,델리리움 트레멘스:15,000원,수도사 ...",https://mp-seoul-image-production-s3.mangoplat...,종로구,['흠….. 아쉽네\n\n흔하지 않은 수도원맥주를 마실 수 있는 곳이다.\n\n그리...


In [41]:
loader =DataFrameLoader(data_frame=df, page_content_column='page_content')
docs = loader.load()
embeddings = OpenAIEmbeddings()

In [42]:
wine_bar_vectorstore = Chroma.from_documents(docs, embeddings)

In [43]:
wine_bar_vectorstore.similarity_search_with_score('여자친구랑 갈만한 와인바', k=5)

[(Document(page_content='[\'여자친구는 여기를 데려가세요.\\n들어가자마자 애인 눈에서 하트가 나오는 걸\\n볼 수 있을걸요. 은은한- 분위기에 사로잡힌\\n해질녘, 압도적인 감성의 와인사케바 #유희\\n⠀\\n제철에 맞는 사시미들이 나오는 [숙성사시미]\\n계절마다 가장 맛있는 부위로만 즐길 수 있고\\n고등어, 광어, 단새우 등 다 맛있었는데 특히\\n짚불 훈연한 삼치는 불향 가득 입에서 녹는다.\\n⠀\\n[해삼내장생면파스타] 바다향 그 자체인 생면\\n파스타를 만났다. 위에는 구운 김까지 올려져\\n고소함 폭발이다. [훈연사시미김부각] 이거는\\n안 시키면 큰일 날 뻔했다고 연발하면서 먹음\\n⠀\\n은은한 조명, 인테리어가 주는 세련된 분위기\\n분주한 셰프님들의 손놀림까지 눈이 즐겁다.\\n퀄리티 높은 안주들과 페어링 할 수 있는 술도\\n굉장히 다양해서 데이트 장소로 아묻따 추천!\', \'음식맛에 반하고 분위기에 취하는 다이닝바\\n\\n성수에 있는 미식가클럽에서 런칭한 세컨 브랜드. 다양한 음식들을 이곳만의 터치를 가미해 특색있는 메뉴들로 탈바꿈해 놓았다. 메뉴판 곳곳에 흥미로운 메뉴들이 넘쳐난다.\\n\\n✔️고노와다파스타\\n고노와다는 해삼내장이라고 한다. 비린맛에 민감한 편이지만 비린맛 하나 없이 고소한 풍미만 맛볼 수 있었다. 위에 올라간 것은 김. 면도 직접 만든 생면을 사용해서 호로록 하고 잘도 넘어간다.\\n\\n✔️안키모빠테와 브레드스틱\\n아귀간빠테를 튀긴 사워도우 위에 올렸다. 크리미한 풍미에 달콤한 아귀간에 튀긴 빵은 맛이 없을래야 없을 수 없다. 맛있는 달콤하고 꼬소한 조합.\\n\\n✔️제철사시미\\n나에게 회는 그저 비리지만 않으면 합격인 음식이라 잘 모르긴 한다. 그래도 착 달라붙는 식감도 좋고 어느하나 비리지 않아서 잘 숙성된 좋은 회라는 생각이 들었다. 구성도 다양하고 다른 메뉴들에 비해서 가성비가 좋게 느껴졌다.\\n\\n적은 양 대비 가격대가 높아서 식사시간대에 오는 것 보다 2차 3차로 

In [44]:
metadata_field_info = [
    AttributeInfo(
        name="price",
        description="The price of the wine bar",
        type="int",
    ),
    AttributeInfo(
        name="rating", 
        description="1-5 rating for the wine bar", 
        type="float"
    ),
    AttributeInfo(
        name="district",
        description="The district of the wine bar.",
        type="str",
    ),
]

In [45]:
# wine_bar_vectorstore = Chroma.from_documents(docs, embeddings)
document_content_description = "Database of a winebar"
llm = OpenAI(temperature=0)
wine_bar_retriever = SelfQueryRetriever.from_llm(
    llm, wine_bar_vectorstore, document_content_description, metadata_field_info=metadata_field_info, verbose=True
)  # Added missing closing parenthesis

#### Tool3: Search in Google

In [46]:
search = SerpAPIWrapper()

#### Tool4: Kakao Map API

In [47]:
import requests

class KakaoMap:
    def __init__(self):
        self.url = 'https://dapi.kakao.com/v2/local/search/keyword.json' 
        self.headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    def run(self, query):
        params = {'query': query,'page': 1} 
        places = requests.get(self.url, params=params, headers=self.headers).json()['documents']
        address = places[0]['address_name']
        if not address.split()[0].startswith('서울'):
            return 'not in seoul'
        else:
            return address.split()[1]

In [48]:
kakao_map = KakaoMap()

In [116]:
tools = [
    Tool(
        name="Wine database",
        func=wine_retriever.get_relevant_documents,
        description="""
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt 20000'
- rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
- wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
- name: The name of wine.
- pairing: The food pairing of wine.
"""
    ),
    Tool(
        name = "Wine bar database",
        func=wine_bar_retriever.get_relevant_documents,
        description="Database about the winebars in Seoul."
"""
- query: The query of winebar. You can search wines with review data like mood or something.
- price: The price range of the wine. 
- rating: 1-5 rating float for the wine. 
- district: The district of winebar. Input district must be korean. For example, if you want to search for wines in Gangnam, enter 'district: 강남구'
"""
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to ask with search. Search in English only."
    ),
    Tool(
        name = "Map",
        func=kakao_map.run,
        description="The tool used to draw a district for a region. When looking for wine bars, you can use this before applying filters based on location."
    ),
]

In [117]:
template = """
Your role is a chatbot that asks customers questions about wine and makes recommendations.
Never forget your name is "이우선".
Keep your responses in short length to retain the user's attention unless you describe the wine for recommendations
Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the user a chance to respond.
Responses should be in Korean.

Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
이우선: the final response to the user

You must respond according to the conversation stage within the triple backticks and conversation history within in '======'.

Current conversation stage: 
```{conversation_stage}```

Conversation history: 
=======
{conversation_history}
=======

Last user saying: {input}
{agent_scratchpad}
"""

conversation_stages_dict = {
    "1": "Introduction: Start the conversation by introducing yourself. Maintain politeness, respect, and a professional tone.",
    "2": "Needs Analysis: Identify the customer's needs to make wine recommendations. Ask one question at a time. Note that the wine database tools are not available. Ask flexible questions, such as the context in which the customer wants to enjoy the wine or what they want to eat with it.",
    "3" :"Checking Price Range: Asking the customer's preferred price point. Again, remember that the tool for this is not available. But if you know the customer's perferences and price range, then search for the three most suitable wines with tool and recommend them in the form of a product card with a vivino link, price, rating, type, a detailed description and image by using img_url.",
    "4" :"Wine Recommendation: Propose the three most suitable wines based on the customer's needs and price range. Each wine recommendation should include an image by using img_url, price, rating, flavor and type, formatted as an appealing product card. Use only wines available in the database for recommendations. If there are no suitable wines in the database, inform the customer. After making a recommendation, inquire whether the customer likes the suggested wine.",
    "5": "Sales: If the customer approves of the recommended wine, provide a detailed description. Supply a product card featuring a link, image, price, and rating. Note that the provided link should open in a new tab when clicked",
    "6" :"Location Suggestions: Recommend wine bars based on a customer's location and mood. Before making a recommendation, always use the map tool to find the district of the customer's preferred location. Then use the wine bar database tool to find a suitable wine bar. Provide the name of the wine bar, along with an image and address.",
    "7": "Concluding the Conversation: Respond appropriately to the customer's comments to wrap up the conversation.",
    "8": "Questions and Answers: This stage involves answering customer's inquiries. Use the search tool or wine database tool to provide specific answers where possible. Describe answer as detailed as possible",
    "9" :"Other Situations: Use this step when the situation does not fit into any of the steps between 1 and 8."
}

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        stage_number = kwargs.pop("stage_number")
        kwargs["conversation_stage"] = conversation_stages_dict[stage_number]
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            if 'Action: Wine bar database' in str(action):
                pattern = re.compile(r'metadata=\{(.*?)\}')
                matches = pattern.findall(f'{observation}')

                result = ', '.join(f'Document(metadata={{{match}}})' for match in matches)
                observation = '[' + result + ']'

            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps", "conversation_history", "stage_number"]
)

In [78]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "이우선: " in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("이우선: ")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

### Define Langchain Agent

In [52]:
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler

In [53]:
llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-4', temperature=0.0), prompt=prompt, verbose=True,)
# llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-4', temperature=0.0, streaming=True, callbacks=[FinalStreamingStdOutCallbackHandler()]), prompt=prompt, verbose=True,)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [54]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

### Gradio

간단하게 웹 구성을 테스트하는 gradio이다. 개선해야할 점이 많지만 맛보기로 올려보았다.

In [55]:
import gradio as gr
import threading
import time
import sys
from typing import List, Union, Optional, Any, Dict

c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
class CustomStreamingStdOutCallbackHandler(FinalStreamingStdOutCallbackHandler):
    """Callback handler for streaming in agents.
    Only works with agents using LLMs that support streaming.

    The output will be streamed until "<END" is reached.
    """
    def __init__(
        self,
        *,
        answer_prefix_tokens: Optional[List[str]] = None,
        end_prefix_tokens: str = "<END",
        strip_tokens: bool = True,
        stream_prefix: bool = False,
        sender: str
    ) -> None:
        """Instantiate EofStreamingStdOutCallbackHandler.

        Args:
            answer_prefix_tokens: Token sequence that prefixes the anwer.
                Default is ["Final", "Answer", ":"]
            end_of_file_token: Token that signals end of file.
                Default is "END"
            strip_tokens: Ignore white spaces and new lines when comparing
                answer_prefix_tokens to last tokens? (to determine if answer has been
                reached)
            stream_prefix: Should answer prefix itself also be streamed?
        """
        super().__init__(answer_prefix_tokens=answer_prefix_tokens, strip_tokens=strip_tokens, stream_prefix=stream_prefix)
        self.end_prefix_tokens = end_prefix_tokens
        self.end_reached = False
        self.sender = sender

    def append_to_last_tokens(self, token: str) -> None:
        self.last_tokens.append(token)
        self.last_tokens_stripped.append(token.strip())
        if len(self.last_tokens) > 5:
            self.last_tokens.pop(0)
            self.last_tokens_stripped.pop(0)

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Run when LLM starts running."""
        self.answer_reached = False
        self.end_reached = False

    def check_if_answer_reached(self) -> bool:
        if self.strip_tokens:
            return ''.join(self.last_tokens_stripped) in self.answer_prefix_tokens_stripped
        else:
            unfied_last_tokens = ''.join(self.last_tokens)
            try:
                unfied_last_tokens.index(self.answer_prefix_tokens)
                return True
            except:
                return False
            
    def check_if_end_reached(self) -> bool:
        if self.strip_tokens:
            return ''.join(self.last_tokens_stripped) in self.answer_prefix_tokens_stripped
        else:
            unfied_last_tokens = ''.join(self.last_tokens)
            # print(unfied_last_tokens)
            try:
                unfied_last_tokens.index(self.end_prefix_tokens)
                self.sender[1] = True
                return True
            except:
                return False

    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        """Run on new LLM token. Only available when streaming is enabled."""
        # Remember the last n tokens, where n = len(answer_prefix_tokens)
        self.append_to_last_tokens(token)
        # Check if the last n tokens match the answer_prefix_tokens list ...
        if not self.answer_reached and self.check_if_answer_reached():
            self.answer_reached = True
            if self.stream_prefix:
                for t in self.last_tokens:
                    sys.stdout.write(t)
                sys.stdout.flush()
            return
        
        if not self.end_reached and self.check_if_end_reached():
            self.end_reached = True

        # ... if yes, then print tokens from now on, unless EOF has been reached
        # print(self.answer_reached, self.end_reached)
        if self.end_reached:
            pass
        elif self.answer_reached:
            if self.last_tokens[-2] == ":":
                pass
            else:
                # global agent_string
                # agent_string += self.last_tokens[-2]
                self.sender[0] += self.last_tokens[-2]
                
                # sys.stdout.write(self.last_tokens[-2])
                # sys.stdout.flush()

In [118]:
class UnifiedAgent:
    def __init__(self):
        sender_for_gradio = ["", False]

        tools = [
    Tool(
        name="Wine database",
        func=wine_retriever.get_relevant_documents,
        description="""
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt 20000'
- rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
- wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
- name: The name of wine.
- pairing: The food pairing of wine.
"""
    ),
    Tool(
        name = "Wine bar database",
        func=wine_bar_retriever.get_relevant_documents,
        description="Database about the winebars in Seoul. Query must be in String"
"""
- query: The query of winebar. You can search wines with review data like mood or something.
- price: The average price of the menu in the wine bar. 
- rating: 1-5 rating float for the reviews.
- district: The district of winebar. Input district must be korean. For example, if you want to search for wines in Gangnam, enter 'district: 강남구'
"""
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to ask with search. Search in English only."
    ),
    Tool(
        name = "Map",
        func=kakao_map.run,
        description="The tool used to draw a district for a region. When looking for wine bars, you can use this before applying filters based on location."
    ),
]

        llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-4', temperature=0.5, streaming=True, callbacks=[CustomStreamingStdOutCallbackHandler(answer_prefix_tokens='이우선:', end_prefix_tokens='<END', strip_tokens=False, sender=sender_for_gradio)]), prompt=prompt, verbose=True,)

        tool_names = [tool.name for tool in tools]
        agent = LLMSingleActionAgent(
            llm_chain=llm_chain, 
            output_parser=output_parser,
            stop=["\nObservation:"], 
            allowed_tools=tool_names
        )
        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=False)

        self.agent_executor = agent_executor
        self.sender = sender_for_gradio


class UnifiedChain:
    def __init__(self):
        stage_analyzer_inception_prompt = load_prompt("./templates/stage_analyzer_inception_prompt_template.json")
        llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)
        stage_analyzer_chain = LLMChain(
            llm=llm,
            prompt=stage_analyzer_inception_prompt, 
            verbose=False, 
            output_key="stage_number")
        
        user_response_prompt = load_prompt("./templates/user_response_prompt.json")
        # 랭체인 모델 선언, 랭체인은 언어모델과 프롬프트로 구성됩니다.
        llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
        user_response_chain = LLMChain(
            llm=llm,
            prompt=user_response_prompt, 
            verbose=False, # 과정을 출력할지
            output_key="user_responses"
        )

        self.stage_analyzer_chain = stage_analyzer_chain
        self.user_response_chain = user_response_chain

In [127]:
with gr.Blocks(css='#chatbot .overflow-y-auto{height:750px}') as demo:
    unified_chain = UnifiedChain()
    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>ChatWine</h1>
            </div>
            <p style="margin-bottom: 10px; font-size: 94%">
                LinkedIn <a href="https://www.linkedin.com/company/audrey-ai/about/">Audrey.ai</a>
            </p>
        </div>""")
    
    chatbot = gr.Chatbot()

    with gr.Row():
        with gr.Column(scale=0.85):
            msg = gr.Textbox()
        with gr.Column(scale=0.15, min_width=0):
            submit_btn = gr.Button("전송")

    user_response_examples = gr.Dataset(samples=[["이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래?"], ["입문자에게 좋은 와인을 추천해줄래?"], ["연인과 가기 좋은 와인바를 알려줘"]], components=[msg], type="index")
    clear_btn = gr.ClearButton([msg, chatbot])

    dev_mod = True
    cur_stage = gr.Textbox(visible=dev_mod, interactive=False, label='current_stage')
    stage_hist = gr.Textbox(visible=dev_mod, value="stage history: ", interactive=False, label='stage history')
    chat_hist = gr.Textbox(visible=dev_mod, interactive=False, label='chatting_history')
    response_examples_text = gr.Textbox(visible=dev_mod, interactive=False, value="이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래?|입문자에게 좋은 와인을 추천해줄래?|연인과 가기 좋은 와인바를 알려줘", label='response_examples')


    def load_example(response_text, input_idx):
        response_examples = []
        for user_response_example in response_text.split('|'):
            response_examples.append([user_response_example])
        return response_examples[input_idx][0]

    def agent_run(agent_exec, inp, sender):
        sender[0] = ""
        agent_exec.run(inp)

    def user_chat(user_message, chat_history_list, chat_history):
        return (chat_history_list + [[user_message, None]], chat_history + f"User: {user_message} <END_OF_TURN>\n", [])

    def bot_stage_pred(user_response, chat_history, stage_history):
        stage_number = unified_chain.stage_analyzer_chain.run({'conversation_history': chat_history, 'stage_history': stage_history, 'last_user_saying':user_response})
        stage_number = stage_number[-1]
        stage_history += stage_number if stage_history == "stage history: " else ", " + stage_number

        return stage_number, stage_history

    def bot_chat(user_response, chat_history, chat_history_list, current_stage): # stream output by yielding
        unified_agent = UnifiedAgent()
        t = threading.Thread(target = agent_run, args=(unified_agent.agent_executor, {'input':user_response, 'conversation_history': chat_history, 'stage_number': current_stage}, unified_agent.sender))
        t.start()

        while(t.is_alive()):
            time.sleep(0.05)
            response = unified_agent.sender[0]
            chat_history_list[-1][1] = response
            yield chat_history_list, chat_history + f"이우선: {response}\n"

        t.join()
        response = unified_agent.sender[0]
        chat_history_list[-1][1] = response
        return chat_history_list, chat_history + f"이우선: {response}\n"

    def bot_response_pred(chat_history):
        response_examples = []
        last_conversation = '<END_OF_TURN>'.join(chat_history.split('<END_OF_TURN>')[-3:])[:-2] + '<END_OF_TURN>\n'
        out = unified_chain.user_response_chain.run({'conversation_history': last_conversation})
        for user_response_example in out.split('|'):
            response_examples.append([user_response_example])
        return [response_examples, out, ""]

    msg.submit(
        user_chat, [msg, chatbot, chat_hist], [chatbot, chat_hist, user_response_examples], queue=False
    ).then(
        bot_stage_pred, [msg, chat_hist, stage_hist], [cur_stage, stage_hist], queue=False
    ).then(
        bot_chat, [msg, chat_hist, chatbot, cur_stage], [chatbot, chat_hist]
    ).then(
        bot_response_pred, chat_hist, [user_response_examples, response_examples_text, msg]
    )

    submit_btn.click(
        user_chat, [msg, chatbot, chat_hist], [chatbot, chat_hist, user_response_examples], queue=False
    ).then(
        bot_stage_pred, [msg, chat_hist, stage_hist], [cur_stage, stage_hist], queue=False
    ).then(
        bot_chat, [msg, chat_hist, chatbot, cur_stage], [chatbot, chat_hist]
    ).then(
        bot_response_pred, chat_hist, [user_response_examples, response_examples_text, msg]
    )
    clear_btn.click(lambda: None, None, chatbot, queue=False)
    user_response_examples.click(load_example, inputs=[response_examples_text, user_response_examples], outputs=[msg])

demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.




> Entering new  chain...
Prompt after formatting:

Your role is a chatbot that asks customers questions about wine and makes recommendations.
Never forget your name is "이우선".
Keep your responses in short length to retain the user's attention unless you describe the wine for recommendations
Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the user a chance to respond.
Responses should be in Korean.

Complete the objective as best you can. You have access to the following tools:

Wine database: 
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt 20000'
- rating: 1-5 rating float for the wine. You have t

c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='크루그 그랑 뀌베 170에디션' filter=None limit=None


> Entering new  chain...
Prompt after formatting:

Your role is a chatbot that asks customers questions about wine and makes recommendations.
Never forget your name is "이우선".
Keep your responses in short length to retain the user's attention unless you describe the wine for recommendations
Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the user a chance to respond.
Responses should be in Korean.

Complete the objective as best you can. You have access to the following tools:

Wine database: 
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt 20000'
- r

In [ ]:
gr.close_all()

In [ ]:
with gr.Blocks(css='#chatbot .overflow-y-auto{height:750px}') as demo:
    
    chatbot = gr.Chatbot()

    
    bot_chat, [msg, chat_hist, chatbot, cur_stage], [chatbot, chat_hist]
demo.launch()    